# "Conv2d to rnn"

In [1]:
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras.regularizers import l2

Using TensorFlow backend.


### Load Data

In [15]:
data_path='../preprocessing/preprocessed_data'

In [16]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=True)
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge'], num=-1, shuffle=True)

In [17]:
callback_stopearly = keras.callbacks.EarlyStopping(monitor='val_acc',
                                                   patience=5)

### Simple First Model

In [35]:
weight_decay = 0.01

model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=32,
                         fmin=0.0, fmax=8000, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
model.add(layers.Conv2D(16, (4, 4), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model.add(layers.MaxPooling2D((2, 2 )))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (2, 2), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model.add(layers.MaxPooling2D((2, 1)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (2, 2), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model.add(layers.MaxPooling2D((2, 1)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, (2, 2), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model.add(layers.MaxPooling2D((2, 1)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, (2, 2), activation='relu', padding='same', kernel_regularizer=l2(weight_decay)))
model.add(layers.MaxPooling2D((2, 1)))
model.add(layers.BatchNormalization())

model.add(layers.Reshape((-1, 64)))
model.add(layers.GRU(64, dropout=0.3))
model.add(layers.Dense(3, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_31 (Melspectr (None, 32, 313, 1)        271392    
_________________________________________________________________
conv2d_136 (Conv2D)          (None, 32, 313, 16)       272       
_________________________________________________________________
max_pooling2d_133 (MaxPoolin (None, 16, 156, 16)       0         
_________________________________________________________________
batch_normalization_125 (Bat (None, 16, 156, 16)       64        
_________________________________________________________________
conv2d_137 (Conv2D)          (None, 16, 156, 32)       2080      
_________________________________________________________________
max_pooling2d_134 (MaxPoolin (None, 8, 156, 32)        0         
_________________________________________________________________
batch_normalization_126 (Bat (None, 8, 156, 32)        128       
__________

=> 94%

### Model 2
derived from https://github.com/YerevaNN/Spoken-language-identification/blob/master/theano/networks/tc_net_rnn_onernn.py

In [ ]:
shared_GRU = layers.CuDNNGRU(15)
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=223, n_hop=256,
                         fmin=0.0, fmax=5500, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
model.add(layers.Conv2D(16, (7, 7), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3 ), strides=(2,2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Permute((3,2,1)))
model.add(layers.TimeDistributed(shared_GRU))
model.add(layers.Reshape((-1,)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(3, activation='softmax'))
#model.summary()

=> 91%

### Model 3: Inception
derived from https://github.com/HPI-DeepLearning/crnn-lid/blob/master/keras/models/inceptionv3_crnn.py

In [42]:
from keras.applications.inception_v3 import InceptionV3

In [68]:
input_tensor = layers.Input(shape=(1, 80000))
x = Melspectrogram(n_dft=512,
                   padding='same', sr=16000, n_mels=128, n_hop=256,
                   fmin=0.0, fmax=5500, power_melgram=1.0,
                   return_decibel_melgram=False, trainable_fb=False,
                   trainable_kernel=False)(input_tensor)

inception_model = InceptionV3(include_top=False, weights=None, input_tensor=x)

x = inception_model.output

#x = GlobalAveragePooling2D()(x)

# (bs, y, x, c) --> (bs, x, y, c)
x = layers.Permute((2, 1, 3))(x)

# (bs, x, y, c) --> (bs, x, y * c)
_a, _y, _c = [int(s) for s in x.shape[1:]]
x = layers.Reshape((_a, _y*_c))(x)
x = layers.wrappers.Bidirectional(layers.LSTM(64, return_sequences=False), merge_mode="concat")(x)
x = layers.Dense(3, activation='softmax')(x)

model = models.Model(input_tensor, outputs=x)
#model.summary()

tooooo biiiigggg...

### Model 3: MobileNet

In [55]:
from keras.applications.mobilenetv2 import MobileNetV2

In [69]:
input_tensor = layers.Input(shape=(1, 80000))
x = Melspectrogram(n_dft=512,
                   padding='same', sr=16000, n_mels=223, n_hop=360,
                   fmin=0.0, fmax=5500, power_melgram=1.0,
                   return_decibel_melgram=False, trainable_fb=False,
                   trainable_kernel=False)(input_tensor)

inception_model = MobileNetV2(include_top=False, weights=None, input_tensor=x,
                            alpha=0.25, pooling=None)

x = inception_model.output

#x = GlobalAveragePooling2D()(x)

# (bs, y, x, c) --> (bs, x, y, c)
x = layers.Permute((2, 1, 3))(x)

# (bs, x, y, c) --> (bs, x, y * c)
_a, _y, _c = [int(s) for s in x.shape[1:]]
x = layers.Reshape((_a, _y*_c))(x)
x = layers.GRU(64, return_sequences=False, dropout=0.5)(x)
x = layers.Dense(3, activation='softmax')(x)

model = models.Model(input_tensor, outputs=x)
#model.summary()

KeyboardInterrupt: 

## Train

In [24]:
model.compile(optimizer='Rmsprop',
              metrics=['accuracy'],
              loss='categorical_crossentropy')

In [25]:
history = model.fit(x=train_data,
                        y=train_labels,
                        batch_size=64, 
                        epochs=20,
                        validation_data=(val_data, val_labels), 
                        shuffle=True,
                        callbacks=[callback_stopearly])

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
17792/50000 [=========>....................] - ETA: 2:43 - loss: 1.0573 - acc: 0.4400

KeyboardInterrupt: 